In [36]:
from somefunctions import *
from f1_score import *
import scipy

In [41]:
# Model parameters
pool_size = (2, 2)
train_shape = 400 #size of the training images
patch_size = 16
input_size = 64
pad_size = int(input_size/2 - patch_size/2)
pad_rotate_size = int( input_size / np.sqrt(2) ) + 2


# Training parameters
reg = 1e-5  #regularization term
learning_rate = 0.001
nb_epoch = 45 #very small, only preliminary tests
batch_size = 250
steps_per_epoch = 125 #the number of training samples is huge, arbitrary value


# Data augmentation parameters
FLIP_FLAG = True # add random flips to the patches
ROTATION_FLAG = True # add random rotation to the patches
BRIGHT_CONTRAST_FLAG = True # modify randomly the brightness and the constrast


#Other stuff
NameWeights = 'NicolaWeights'
SubmissionName = 'NicolaSubmission.csv'

In [47]:
def data_augmentation(X):
    '''Data augmentation on X, element of size (input_size * input_size * 3)'''
    #flip
    if FLIP_FLAG:
        flip_decision = np.random.choice(3)
        if flip_decision == 1:
            X = np.flipud(X)
        if flip_decision == 2: 
            X = np.fliplr(X)
    
    #contrast and brightness
    if BRIGHT_CONTRAST_FLAG:
        brightness = np.random.rand()*0.3 - 0.15
        contrast = np.random.rand()*0.25 - 0.125
        X = np.clip( X * (contrast/0.5+1) - contrast + brightness, 0, 1)
        
    return X

def crop_center(img,cropx,cropy):
    if len(img.shape) == 3:
        y,x, _ = img.shape
    if len(img.shape) == 2:
        y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def generate_minibatch_with_arbitrary_rotation(X,Y):
    """
    Generate a minibatch
    """
    while 1:        
        # Generate one minibatch
        X_batch = np.empty((batch_size, input_size, input_size, 3))
        Y_batch = np.empty(batch_size)
        low=pad_rotate_size + patch_size // 2
        high = pad_rotate_size + train_shape - patch_size // 2
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            
            x_coord = np.random.randint(low=low, high = high ) 
            y_coord = np.random.randint(low=low, high = high )
            
            X_temp = X[idx,x_coord - pad_rotate_size:x_coord + pad_rotate_size,
                           y_coord - pad_rotate_size:y_coord + pad_rotate_size]
            
            #arbitrary rotation and crop of X_temp
            degree = np.random.choice(180)
            X_temp = scipy.ndimage.interpolation.rotate(X_temp, degree)
            X_temp = crop_center(X_temp,input_size,input_size)
            
            X_temp = data_augmentation(X_temp)
            
            X_batch[i] = X_temp
            
            gt_temp = Y[idx,x_coord - pad_rotate_size:x_coord + pad_rotate_size,
                            y_coord - pad_rotate_size:y_coord + pad_rotate_size]  #TODO: reduce this crop size
            
            #arbitrary rotation and crop of gt_temp
            #same degree
            gt_temp = scipy.ndimage.interpolation.rotate(gt_temp, degree)
            gt_temp = crop_center(gt_temp,patch_size,patch_size)
            Y_batch[i] = patch_to_label(gt_temp)
            
        yield X_batch, Y_batch
        
        
        
        

In [43]:
def create_model():
    '''Create a sequential model'''        
    model = Sequential()
    
    model.add(Convolution2D(64, (5,5), 
                            input_shape = ( input_size, input_size, 3),
                            padding = 'SAME', activation = 'relu',
                            kernel_initializer = K_init.RandomUniform(minval=-0.05, maxval=0.05, seed=1)
                           ))
    
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Convolution2D(64, (3,3), 
                            input_shape = ( input_size, input_size, 3),
                            padding = 'SAME', activation = 'relu',
                            kernel_initializer = K_init.RandomUniform(minval=-0.05, maxval=0.05, seed=1)
                           ))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Convolution2D(128, (3,3),
                            padding = 'SAME', activation = 'relu',
                            kernel_initializer = K_init.RandomUniform(minval=-0.05, maxval=0.05, seed=1)
                           ))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Convolution2D(256, (3,3),
                            padding = 'SAME', activation = 'relu',
                            kernel_initializer = K_init.RandomUniform(minval=-0.05, maxval=0.05, seed=1)
                           ))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', kernel_regularizer = l2(reg)))
    model.add(Dropout(0.5))         
    model.add(Dense(256, activation = 'relu', kernel_regularizer = l2(reg)))
    model.add(Dropout(0.5))       
    model.add(Dense(units = 1, activation = 'sigmoid'))

    #Optimizer          
    opt = Adam(lr=learning_rate) # Adam optimizer with default initial learning rate
 

    # This callback reduces the learning rate when the training accuracy does not improve any more
    lr_callback = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=5,
                                    verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
    
    # Stops the training process upon convergence
    stop_callback = EarlyStopping(monitor='acc', min_delta=0.0001, patience=10, verbose=1, mode='auto')
    
    model.compile(loss=binary_crossentropy,
                  optimizer=opt,
                  metrics=['acc', f1_score])
    
    return model, stop_callback, lr_callback

    

def train(X, Y):    
    '''
    Generate an instance of the model an train the model on X, Y
    '''
    print('Training set shape: ', X.shape) 
    print(f'Batch_size: {batch_size} \nSteps per epoch: {steps_per_epoch} \n')
    
    
    model, stop_callback, lr_callback = create_model()
    
    np.random.seed(20122018) # Reproducibility + remember the deadline is the 20.12.2018
    
    try:
        model.fit_generator(generate_minibatch_with_arbitrary_rotation(X,Y),
                            steps_per_epoch=steps_per_epoch,
                            nb_epoch=nb_epoch,
                            verbose=1,
                            callbacks=[lr_callback, stop_callback])
    except KeyboardInterrupt:
        print('\n\nKeyboard interruption!\n\n')
        pass
    

    model.save_weights(NameWeights)
    
    print(f'Training completed, weights saved in: {NameWeights}')
    
    return model

In [44]:
# Load a set of image
root_dir = "../Data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 
print("Loading " + str(n) + " images")
imgs_original = [load_image(image_dir + files[i]) for i in range(n)]
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
gt_imgs_not_padded = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

Loading 100 images
satImage_001.png
Loading 100 images
satImage_001.png


In [45]:
imgs = padding_imgs(np.array(imgs),pad_rotate_size)
gt_imgs = padding_GT(np.array(gt_imgs),pad_rotate_size)
print(imgs.shape)
print(gt_imgs.shape)

(100, 494, 494, 3)
(100, 494, 494)


In [48]:
model = train(imgs,gt_imgs)

Training set shape:  (100, 494, 494, 3)
Batch_size: 250 
Steps per epoch: 125 



C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=125, verbose=1, callbacks=[<keras.ca..., epochs=45)`


Epoch 1/45
 15/125 [==>...........................] - ETA: 9:48 - loss: 0.6892 - acc: 0.7355 - f1_score: 0.0169

Keyboard interruption!


Training completed, weights saved in: NicolaWeights


In [49]:
IDX = 1
X = imgs_to_windows(imgs,400,patch_size,input_size)
patches_idx = X[625*IDX:625*(IDX+1)]

print(X.shape)
print(patches_idx.shape)

(62500, 64, 64, 3)
(625, 64, 64, 3)


In [50]:
Z_idx = model.predict(patches_idx)
labels_idx = (Z_idx > 0.5) *1 
print(Z_idx[:3])    

[[0.20874691]
 [0.20507324]
 [0.20845413]]
